# **Exercise session 3: Convolutional neural network (CNN) and support vector machine (SVM) to detect $CH_4$ emissions from satellite data**

## Exercise 3.3 Batch jobs and server-side computations using OpenEO (1 pt)

So far you have been using openEO for synchronous execution of your requests: you submitted a request and the result came as a direct response to your request. This is not feasible for heavier work. Instead, one should submit the requests as batch jobs. Familiarise yourself with the Batch-job-combine-tasks.ipynb file.

Furthermore, instead of downloading large amounts of raw data, one can process the data directly on server side. You can check all available processes by running 'connection.list_processes()'. For more details, see the [EO Cookbook](https://openeo.org/documentation/1.0/cookbook/#temporal-mean-reduce-dimension). Note that they give two alternatives on how to compute the temporal mean and we recommend to use the 'reduce_dimension' function.

Task: fix an area of interest and a time period and load the min, max, mean and standard deviation of CH4. Perform this task as a batch job using server-side processes. You should save the output in the JSON format. Note that the output is computed per pixel.

In [2]:
import pathlib
import rasterio
import matplotlib.pyplot as plt
import openeo
import pandas as pd
import xarray as xr
import os
import joblib
import json

In [15]:
connection = openeo.connect(url="openeo.dataspace.copernicus.eu")
connection.authenticate_oidc_device()

Visit https://identity.dataspace.copernicus.eu/auth/realms/CDSE/device?user_code=WPIX-JSZV 📋 to authenticate.

[####################################-] ✅ Authorized successfully

<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [16]:
# print(connection.list_collections())
for collection in connection.list_collections():
    print(collection['id'])

SENTINEL3_OLCI_L1B
SENTINEL3_SLSTR
SENTINEL_5P_L2
SENTINEL2_L1C
SENTINEL2_L2A
SENTINEL1_GRD
COPERNICUS_30
LANDSAT8_L2


In [17]:
connection.list_jobs()

[{'costs': 13.0,
  'created': '2024-01-27T13:12:51Z',
  'id': 'j-240127b8558449a48882d6624536c92d',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:12:51Z'},
 {'costs': 12.0,
  'created': '2024-01-27T13:18:52Z',
  'id': 'j-24012709537241da9e03d96b39dd21c4',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:18:52Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:42:16Z',
  'id': 'j-240127b7529946509de4963ae7053754',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:42:16Z'},
 {'costs': 2.0,
  'created': '2024-01-27T11:13:17Z',
  'id': 'j-240127d1953146d5aac31aaa12a658d1',
  'status': 'error',
  'title': 'CH4 std',
  'updated': '2024-01-27T11:13:17Z'},
 {'costs': 3.0,
  'created': '2024-01-27T13:48:17Z',
  'id': 'j-2401277667e7482db316a681107a3bad',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:48:17Z'},
 {'costs': 3.0,
  'created': '2024-01-27T11:05:43Z',
  'id': 'j-240127e98f9f45c595548674c49365af',
  'status': 'finished',
  'title': 'CH4 min',
  'updated': '2024-01-27T11:05:43Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:40:42Z',
  'id': 'j-2401278c48544638b14c10bf4d46ffc9',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:40:42Z'},
 {'costs': 5.0,
  'created': '2024-01-27T13:56:32Z',
  'id': 'j-2401274ab5f74818b876ecdf9962672a',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:56:32Z'},
 {'costs': 4.0,
  'created': '2024-02-12T21:56:51Z',
  'id': 'j-240212aba8704a71b03a5610d2db6f30',
  'status': 'finished',
  'title': 'CH4 std',
  'updated': '2024-02-12T21:59:18Z'},
 {'costs': 5.0,
  'created': '2024-01-27T13:31:31Z',
  'id': 'j-240127d0e00b442bb27e5720e491cdd4',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:31:31Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:12:21Z',
  'id': 'j-240127401c174a8494e0d7d3a0cf3b50',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:12:21Z'},
 {'costs': 3.0,
  'created': '2024-02-12T23:33:40Z',
  'id': 'j-2402127ed08448d79ebef06b4332d521',
  'status': 'finished',
  'title': 's5_CH4_aggregation',
  'updated': '2024-02-12T23:36:16Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:28:11Z',
  'id': 'j-240127a4d8294d75bffd5eb762eee5a9',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:28:11Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:18:13Z',
  'id': 'j-2401275dd65441bda216319c2e6daac0',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:18:13Z'},
 {'costs': 3.0,
  'created': '2024-02-12T23:47:01Z',
  'id': 'j-240212264b5145b784aea23bd501dc23',
  'status': 'finished',
  'title': 's5_CH4_aggregation',
  'updated': '2024-02-12T23:49:13Z'},
 {'created': '2024-01-27T11:54:45Z',
  'id': 'j-240127baaca94b28be032d58edb5bb77',
  'status': 'error',
  'title': 'CH4 min',
  'updated': '2024-01-29T12:01:36Z'},
 {'costs': 4.0,
  'created': '2024-02-12T21:16:56Z',
  'id': 'j-2402120d0fd44e6887370dec154fb9f7',
  'status': 'finished',
  'title': 'CH4 max',
  'updated': '2024-02-12T21:19:36Z'},
 {'costs': 3.0,
  'created': '2024-01-27T09:20:16Z',
  'id': 'j-2401274983774b1d94dcfead78fc58a5',
  'status': 'finished',
  'title': 'CH4 Analysis',
  'updated': '2024-01-27T09:20:16Z'},
 {'costs': 4.0,
  'created': '2024-02-12T21:19:55Z',
  'id': 'j-24021229d6b64ed3877fe10dbd35eb72',
  'status': 'finished',
  'title': 'CH4 mean',
  'updated': '2024-02-12T21:22:28Z'},
 {'costs': 3.0,
  'created': '2024-01-27T13:25:13Z',
  'id': 'j-2401272c6d054d099f7574c67105cb24',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:25:13Z'},
 {'costs': 5.0,
  'created': '2024-01-27T14:36:40Z',
  'id': 'j-2401278eee69442eb02a329fb9194043',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:36:40Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:00:31Z',
  'id': 'j-24012718c3f84551b27059519744abe3',
  'status': 'finished',
  'title': 're

In [18]:
# Load the coordinates and labels of the CH4 plumes coming from gas infrastructure, 
# oil infrastruture and coal mines from [all TROPOMI detected plumes for 2021. (Schuit et al. 2023)]
# (https://zenodo.org/records/8087134). 
download_path = "/coursedata/users/nguyenb5"

CH4_plumes = pd.read_csv(f"{download_path}/assignment3_Task4/Schuit_etal2023_TROPOMI_all_plume_detections_2021.csv")

# Filter out rows where estimated_source_type is "unclassified"
CH4_plumes = CH4_plumes[CH4_plumes['estimated_source_type'] != 'Unclassified']

# For each plume, compute the spatial extent of an area 1 deg x 1 deg around the plume
CH4_plumes['west'] = CH4_plumes['lon'] - 0.5
CH4_plumes['east'] = CH4_plumes['lon'] + 0.5
CH4_plumes['south'] = CH4_plumes['lat'] - 0.5
CH4_plumes['north'] = CH4_plumes['lat'] + 0.5

print("Number of data points:", len(CH4_plumes))

CH4_plumes.head()

Number of data points: 2944


,date,time_UTC,lat,lon,source_rate_t/h,uncertainty_t/h,estimated_source_type,west,east,south,north
0,20210101,06:00:45,36.75,109.76,32,16,Coal,109.26,110.26,36.25,37.25
1,20210101,06:00:55,37.53,110.75,39,22,Coal,110.25,111.25,37.03,38.03
2,20210101,07:37:49,20.89,85.22,4,2,Coal,84.72,85.72,20.39,21.39
3,20210101,07:38:15,23.30,90.79,51,15,Landfill/Urban,90.29,91.29,22.80,23.80
4,20210101,07:38:27,23.56,86.44,25,12,Coal,85.94,86.94,23.06,24.06


**1. Define the datacube**

In [20]:
period=("2021-01-01", "2021-01-31")

s5_CH4 = connection.load_collection(
    "SENTINEL_5P_L2",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["CH4"] #as before, server-side computations for S5 accept only one band at a time
)

s5_CO = connection.load_collection(
    "SENTINEL_5P_L2",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["CO"] #as before, server-side computations for S5 accept only one band at a time
)

s5_SO2 = connection.load_collection(
    "SENTINEL_5P_L2",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["SO2"] #as before, server-side computations for S5 accept only one band at a time
)


s5_NO2 = connection.load_collection(
    "SENTINEL_5P_L2",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["NO2"] #as before, server-side computations for S5 accept only one band at a time
)

s3_all_bands = connection.load_collection(
    "SENTINEL3_SLSTR",
    #spatial_extent=optional_bounding_box_for_all_plume_rectangles,
    temporal_extent = period,
    bands=["S5", "S6"] #as before, server-side computations for S3 accept only several bands at a time
)
    

**3. Add reducer and initiate the job**

### Divide and conquer between 5 accounts
### Each account can run 2 concurrent jobs

In [31]:
connection.list_jobs()
# You can manually edit and delete jobs by this link 
# https://editor.openeo.org/?server=openeo.dataspace.copernicus.eu

[{'costs': 13.0,
  'created': '2024-01-27T13:12:51Z',
  'id': 'j-240127b8558449a48882d6624536c92d',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:12:51Z'},
 {'costs': 12.0,
  'created': '2024-01-27T13:18:52Z',
  'id': 'j-24012709537241da9e03d96b39dd21c4',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:18:52Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:42:16Z',
  'id': 'j-240127b7529946509de4963ae7053754',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:42:16Z'},
 {'costs': 2.0,
  'created': '2024-01-27T11:13:17Z',
  'id': 'j-240127d1953146d5aac31aaa12a658d1',
  'status': 'error',
  'title': 'CH4 std',
  'updated': '2024-01-27T11:13:17Z'},
 {'costs': 3.0,
  'created': '2024-01-27T13:48:17Z',
  'id': 'j-2401277667e7482db316a681107a3bad',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:48:17Z'},
 {'costs': 3.0,
  'created': '2024-01-27T11:05:43Z',
  'id': 'j-240127e98f9f45c595548674c49365af',
  'status': 'finished',
  'title': 'CH4 min',
  'updated': '2024-01-27T11:05:43Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:40:42Z',
  'id': 'j-2401278c48544638b14c10bf4d46ffc9',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:40:42Z'},
 {'costs': 5.0,
  'created': '2024-01-27T13:56:32Z',
  'id': 'j-2401274ab5f74818b876ecdf9962672a',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:56:32Z'},
 {'costs': 4.0,
  'created': '2024-02-12T21:56:51Z',
  'id': 'j-240212aba8704a71b03a5610d2db6f30',
  'status': 'finished',
  'title': 'CH4 std',
  'updated': '2024-02-12T21:59:18Z'},
 {'costs': 5.0,
  'created': '2024-01-27T13:31:31Z',
  'id': 'j-240127d0e00b442bb27e5720e491cdd4',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:31:31Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:12:21Z',
  'id': 'j-240127401c174a8494e0d7d3a0cf3b50',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:12:21Z'},
 {'costs': 3.0,
  'created': '2024-02-12T23:33:40Z',
  'id': 'j-2402127ed08448d79ebef06b4332d521',
  'status': 'finished',
  'title': 's5_CH4_aggregation',
  'updated': '2024-02-12T23:36:16Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:28:11Z',
  'id': 'j-240127a4d8294d75bffd5eb762eee5a9',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:28:11Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:18:13Z',
  'id': 'j-2401275dd65441bda216319c2e6daac0',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:18:13Z'},
 {'costs': 3.0,
  'created': '2024-02-12T23:47:01Z',
  'id': 'j-240212264b5145b784aea23bd501dc23',
  'status': 'finished',
  'title': 's5_CH4_aggregation',
  'updated': '2024-02-12T23:49:13Z'},
 {'created': '2024-01-27T11:54:45Z',
  'id': 'j-240127baaca94b28be032d58edb5bb77',
  'status': 'error',
  'title': 'CH4 min',
  'updated': '2024-01-29T12:01:36Z'},
 {'costs': 4.0,
  'created': '2024-02-12T21:16:56Z',
  'id': 'j-2402120d0fd44e6887370dec154fb9f7',
  'status': 'finished',
  'title': 'CH4 max',
  'updated': '2024-02-12T21:19:36Z'},
 {'costs': 3.0,
  'created': '2024-01-27T09:20:16Z',
  'id': 'j-2401274983774b1d94dcfead78fc58a5',
  'status': 'finished',
  'title': 'CH4 Analysis',
  'updated': '2024-01-27T09:20:16Z'},
 {'costs': 4.0,
  'created': '2024-02-12T21:19:55Z',
  'id': 'j-24021229d6b64ed3877fe10dbd35eb72',
  'status': 'finished',
  'title': 'CH4 mean',
  'updated': '2024-02-12T21:22:28Z'},
 {'costs': 3.0,
  'created': '2024-01-27T13:25:13Z',
  'id': 'j-2401272c6d054d099f7574c67105cb24',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T13:25:13Z'},
 {'costs': 5.0,
  'created': '2024-01-27T14:36:40Z',
  'id': 'j-2401278eee69442eb02a329fb9194043',
  'status': 'finished',
  'title': 'reducing',
  'updated': '2024-01-27T14:36:40Z'},
 {'costs': 3.0,
  'created': '2024-01-27T14:00:31Z',
  'id': 'j-24012718c3f84551b27059519744abe3',
  'status': 'finished',
  'title': 're

#### Account 2: Cube s5_CO

In [35]:
datacube_name = "s5_CO"
datacube = s5_CO
plume_type = "gas"

print(f"Starting aggregating {datacube_name}_{plume_type}.json")

with open(f"{download_path}/assignment3_Task4/{plume_type}_geojson.json", 'r') as file:
    plumetype_geojson = json.load(file)

print(f"Number of area of interests for {plume_type}_geojson.json: {len(plumetype_geojson['features'])}")

aggregation = datacube.aggregate_spatial(
    geometries=plumetype_geojson,
    reducer="mean",
)
aggregation = aggregation.save_result(format="JSON")
job=aggregation.create_job(title="aggregation")
job.start_and_wait()
results = job.get_results()
results.download_file(f"{download_path}/assignment3_Task4/{datacube_name}_{plume_type}.json")
print(f"Finish aggregating {datacube_name}_{plume_type}.json")

Starting aggregating s5_CO_gas.json
Number of area of interests for gas_geojson.json: 200
0:00:00 Job 'j-240213837a5547d3a0bd88ee4d00e962': send 'start'
0:00:22 Job 'j-240213837a5547d3a0bd88ee4d00e962': created (progress N/A)
0:00:27 Job 'j-240213837a5547d3a0bd88ee4d00e962': created (progress N/A)
0:00:43 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:00:56 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:01:11 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:01:24 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:01:53 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:02:26 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:02:52 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:03:22 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:04:00 Job 'j-240213837a5547d3a0bd88ee4d00e962': running (progress N/A)
0:04:51 Job 'j-240213837a5547d3a0bd88ee4d00e

In [ ]:
datacube_name = "s5_CO"
datacube = s5_CO
plume_type = "coal"

print(f"Starting aggregating {datacube_name}_{plume_type}.json")

with open(f"{download_path}/assignment3_Task4/{plume_type}_geojson.json", 'r') as file:
    plumetype_geojson = json.load(file)
    
aggregation = datacube.aggregate_spatial(
    geometries=plumetype_geojson,
    reducer="mean",
)
aggregation = aggregation.save_result(format="JSON")
job=aggregation.create_job(title="aggregation")
job.start_and_wait()
results = job.get_results()
results.download_file(f"{download_path}/assignment3_Task4/{datacube_name}_{plume_type}.json")
print(f"Finish aggregating {datacube_name}_{plume_type}.json")

In [ ]:
datacube_name = "s5_CO"
datacube = s5_CO
plume_type = "oil"

print(f"Starting aggregating {datacube_name}_{plume_type}.json")

with open(f"{download_path}/assignment3_Task4/{plume_type}_geojson.json", 'r') as file:
    plumetype_geojson = json.load(file)
    
aggregation = datacube.aggregate_spatial(
    geometries=plumetype_geojson,
    reducer="mean",
)
aggregation = aggregation.save_result(format="JSON")
job=aggregation.create_job(title="aggregation")
job.start_and_wait()
results = job.get_results()
results.download_file(f"{download_path}/assignment3_Task4/{datacube_name}_{plume_type}.json")
print(f"Finish aggregating {datacube_name}_{plume_type}.json")